In [1]:
%pwd

'c:\\Users\\lenovo\\Desktop\\Stage\\text-to-3D_Model_Generation\\research'

In [2]:
import os
os.chdir("..")
%pwd

'c:\\Users\\lenovo\\Desktop\\Stage\\text-to-3D_Model_Generation'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TextEmbeddingConfig:
    local_data_file: Path
    embedding_dir: Path
    model_name: str
    cache_dir: Path

In [5]:
from textTo3DModelGen.constants import *
from textTo3DModelGen.utils.common import read_yaml, create_directories

c:\Users\lenovo\Desktop\Stage\text-to-3D_Model_Generation\.venv37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = HYPER_PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([
            self.config.artifacts_root
        ])

    def get_text_embedding_config(self) -> TextEmbeddingConfig:
        config = self.config.text_embedding

        text_embedding_config = TextEmbeddingConfig(
            local_data_file = config.local_data_file,
            embedding_dir = config.embedding_dir,
            model_name = config.model_name,
            cache_dir = config.cache_dir
        )

        return text_embedding_config


In [8]:
from textTo3DModelGen import logger
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
import torch

In [11]:
class TextEmbedding:
    def __init__(self, config: TextEmbeddingConfig):
        self.config = config
        
    # Function to convert text to latent vectors
    def text_to_latent_vector(self, texts, model, processor):
        # Preprocess the texts
        inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)
        # Forward pass through the model
        with torch.no_grad():
            outputs = model.get_text_features(**inputs)
        return outputs

    def embedding_all(self):
        try:
            data = pd.read_csv(self.config.local_data_file)
            logger.info(f"Read of data from {self.config.local_data_file} is done.")

            model = CLIPModel.from_pretrained(self.config.model_name, cache_dir=self.config.cache_dir)
            logger.info(f"Load CLIP Model {self.config.model_name} to {self.config.cache_dir} is done.")

            processor = CLIPProcessor.from_pretrained(self.config.model_name, cache_dir=self.config.cache_dir)
            logger.info(f"Load CLIP Processor {self.config.model_name} to {self.config.cache_dir} is done.")

            for index, row in data.iterrows():
                model_id = row['uids']
                description = row[' description']

                # Ensure the model_id folder exists in the embedding directory
                model_embedding_dir = os.path.join(self.config.embedding_dir, model_id)
                create_directories([model_embedding_dir])

                text_embedding = self.text_to_latent_vector(description, model, processor)

                # Save the embedding
                path_to_save_emb = os.path.join(model_embedding_dir, 'condition.pt')
                torch.save(text_embedding.squeeze(0), path_to_save_emb)
                logger.info(f"Text embedding of model {model_id} is saved to {path_to_save_emb}.")
        except Exception as e:
            raise e

In [12]:
try:
    config = ConfigurationManager()
    text_embedding_config = config.get_text_embedding_config()
    text_embedding = TextEmbedding(config= text_embedding_config)
    text_embedding.embedding_all()
except Exception as e:
    raise e

[2024-09-02 14:17:24,115: INFO: yaml file: config\config.yaml loaded successfully]
[2024-09-02 14:17:24,128: INFO: yaml file: hyper_params.yaml loaded successfully]
[2024-09-02 14:17:24,133: INFO: created directory at: artifacts]
[2024-09-02 14:17:24,209: INFO: Read of data from artifacts/data_ingestion/objaverse_with_description.csv is done.]
[2024-09-02 14:17:31,257: INFO: Load CLIP Model openai/clip-vit-base-patch32 to models is done.]
[2024-09-02 14:17:31,770: INFO: Load CLIP Processor openai/clip-vit-base-patch32 to models is done.]
[2024-09-02 14:17:31,772: INFO: created directory at: artifacts/dataset/embedding\ed51a51909ee46c780db3a85e821feb2]
[2024-09-02 14:17:31,859: INFO: Text embedding of model ed51a51909ee46c780db3a85e821feb2 is saved to artifacts/dataset/embedding\ed51a51909ee46c780db3a85e821feb2\condition.pt.]
[2024-09-02 14:17:31,862: INFO: created directory at: artifacts/dataset/embedding\9110b606f6c547b2980fcb3c8c4b6a1c]
[2024-09-02 14:17:31,982: INFO: Text embedding 